In [9]:
import pandas
import h5py

In [10]:
class espion_file:
    """Loader for erg ESPION CSV files into Python"""
    def __init__(self, filepath, filename, species):
        self.basedir = "/Users/angueyraaristjm/Documents/LiData/invivoERG/"
        self.filepath = filepath
        self.filename = filename
        self.savepath = self.basedir + self.filepath + "/"
        self.fullpath = self.savepath + self.filename + ".csv"
        self.species = species
        self.metadata = self.pull_metadata()
        self.datatable = self.pull_datatable()
        self.data = self.pull_data()
        self.HDF5remap()
    
    def pull_metadata(self):
        # pull and parse metadata information
        csvparams = pandas.read_csv(self.fullpath, header=1, usecols=[0, 1], nrows=10)
        csvparams = csvparams.dropna()
        metadata = dict()
        intfields = ["Steps", "Channels"]
        datefields = ["DOB", "Date performed"]
        for i in range(1, 10):
            if csvparams.Parameter[i] in intfields:
                metadata[csvparams.Parameter[i]] = int(csvparams.Value[i])
            elif csvparams.Parameter[i] in datefields:
                metadata[csvparams.Parameter[i]] = pandas.to_datetime(csvparams.Value[i])
            elif csvparams.Parameter[i] == "Family Name":
                metadata["ID"] = csvparams.Value[i]
            else: 
                metadata[csvparams.Parameter[i]] = csvparams.Value[i]
        metadata['Species'] = self.species
        return metadata
                
    def pull_datatable(self):
        # pull datatable to parse data
        fullcsv = pandas.read_csv(self.fullpath, header=0)
        if "Data Table" in fullcsv:
            # print("Data Table is Right")
            datatable = pandas.read_csv(self.fullpath, header=1, usecols=[3, 4, 5, 8])
            datatable = datatable.dropna()
            datatable = datatable.astype(int)
        if fullcsv.ix[12, 0] == "Data Table":
            # print("Data Table is Below")
            datatable = pandas.read_csv(self.fullpath, header=1, usecols=[0, 1, 2, 5], skiprows=13)
            datatable = datatable.dropna()
            datatable = datatable.astype(int)
        return datatable
    
    def pull_data(self):
        # parse data based on data table
        fullcsv = pandas.read_csv(self.fullpath, header=0)
        data = dict()
        for step in range(self.metadata['Steps']):
            stepname = "Step" + str(step+1).zfill(2)
            # print(stepname)
            ch1start = self.datatable.Column[(self.datatable.Step==(int(step+1))) & (self.datatable.Chan==1)]
            ch2start = self.datatable.Column[(self.datatable.Step==(int(step+1))) & (self.datatable.Chan==2)]
            ntrials = self.datatable.Trials[(self.datatable.Step==(int(step+1))) & (self.datatable.Chan==1)]
            if len(ch1start)==1:
                #normally each step runs only once but if it's repeated, ESPION doubles the entries
                ch1start = int(ch1start)
                ch2start = int(ch2start-1)
                ntrials = int(ntrials)
            elif len(ch1start.unique())==1:
                #found duplicates but all have the same column start
                ch1start = int(ch1start.unique())
                ch2start = int(ch2start.unique()-1)
                ntrials = int(ntrials.sum())
            data[stepname] = self.espion_step(ch1start=ch1start, ch2start=ch2start, ntrials=ntrials, csvtable=fullcsv)
        return data
    
    @staticmethod
    def espion_step(ch1start, ch2start, ntrials, csvtable):
        """Loader for a single erg ESPION step"""
        colstart = ch1start-1
        colend = colstart+1+(ntrials*2)
        currcsv = csvtable.ix[0:, colstart:colend].copy(deep=0)
        currcsv = currcsv.dropna().reset_index(drop=True)
        currcsv = currcsv.drop(0).reset_index(drop=True)
        colnames = []
        ch1cnt = 0
        ch2cnt = 0
        for i in range(0, len(currcsv.columns)):
            currcsv.ix[0:, i] = pandas.to_numeric(currcsv.ix[0:, i])
            if i == 0:
                colnames.append('t')
            elif 1 <= i < 1+ntrials:
                ch1cnt += 1
                colnames.append('L' + str(ch1cnt).zfill(2))
            elif 1+ntrials <= i < 1+(ntrials*2):
                ch2cnt += 1
                colnames.append('R' + str(ch2cnt).zfill(2))
        currcsv.columns = colnames
        currcsv = currcsv.divide(1000)
        csvoutput = currcsv.copy()
        return csvoutput

    def HDF5remap(self):
        dt = h5py.special_dtype(vlen=bytes)
        intfields = ["Steps", "Channels"]
        
        h5name = self.savepath + self.filename + ".h5"
        # print('\nSaving h5 file...')
        with h5py.File(h5name, 'w') as hfile:
#             print('\tFrom datatable:')
            for col in self.datatable.columns:
                hfile.create_dataset(col.replace(' ','_'), data=self.datatable.get(col))
#                 print('\t\t'+ col)
#             print('\tFrom metadata:')
            for key in self.metadata:
                if key in intfields:
                    hfile.attrs.create(key.replace(' ','_'), data=self.metadata[key])
                else:
                    hfile.attrs.create(key.replace(' ','_'), data=str(self.metadata[key]), dtype=dt)
#                 print('\t\t' + key)
            # print('\tFrom data:')
            for step in self.data:
                group = hfile.create_group(step)
                group.create_dataset('t', data=self.data[step].filter(regex = 't'))
                group.create_dataset('L', data=self.data[step].filter(regex = 'L'))
                group.create_dataset('R', data=self.data[step].filter(regex = 'R'))
                # print('\t\t' + step)
        print('Saved to: ' + h5name)

In [12]:
a = espion_file("20160422", "01_1sG", "Squirrel")
# a = espion_file("20160422", "02_IseriesXe", "Squirrel")
# print("a.datatable is " + str(len(a.datatable.axes[0])) + " x " + str(len(a.datatable.axes[1])))
#b = espion_file("20160422", "02_IseriesXe", "Squirrel")
#print("b.datatable is " + str(len(b.datatable.axes[0])) + " x " + str(len(b.datatable.axes[1])))

Saved to: /Users/angueyraaristjm/Documents/LiData/invivoERG/20160422/01_1sG.h5


In [ ]:
print(a.metadata)

# print(str(format(aa,'%2f'))
# print(a.datatable)
# print(a.data['Step2'].head())
# print(a.data['Step1'].filter(regex = "R"))

In [6]:
s=str('Data p').replace(' ','_')
print(s)

Data_p


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, ToggleButton, Checkbox

plt.style.use('ggplot')
%matplotlib inline
%matplotlib notebook
matplotlib.rc('axes', labelcolor='white') 
matplotlib.rc('xtick', color='white')
matplotlib.rc('ytick', color='white')
matplotlib.rc('font', size=8)
matplotlib.rc('figure', dpi=100)

In [ ]:
fig = plt.figure(figsize=(6, 5))
aH = fig.add_subplot(111)
#fig = plt.figure()
#aH = fig.add_axes([0.01, 0.01, 1.0, 1.0]) # left, bottom, width, height (range 0 to 1)
aH.title.set_color('white')

aH.set_title("1sG")
aH.set_ylabel("Potential ($\mu$V)")
aH.set_xlabel("Time (s)")

def plot_trial(pdcsv, axishandle, trialname, pbool):
    pdcsv.data.currcsv.plot(x="t", y=trialname, ax=axishandle, ls=":", alpha=pbool, label=trialname)
    return axishandle.lines[-1]

lH1 = plot_trial(a, aH, "L1_1", True)
lH2 = plot_trial(a, aH, "L1_2", True)
lH3 = plot_trial(a, aH, "L1_3", True)
#lH4 = plot_trial(a, aH, "L1_4", True)
#lH5 = plot_trial(a, aH, "L1_5", True)

# alpha_slider = FloatSlider(min=0, max=1, step=.1, value=5)
# @interact(alpha=alpha_slider)
# def modalpha(alpha):
#     print(alpha)
#     lH1.lines[0]set_alpha(alpha)
#a.data.currcsv.plot(x="t", y="R1_1", ax=aH, ls=":")


tme = a.data.currcsv.t
mean = a.data.currcsv[["L1_1", "L1_2", "L1_3", "L1_4", "L1_5"]].mean(axis=1)
lH = aH.plot(tme, mean, label="Mean", color=[0, 0, 0], lw=2)

In [ ]:
selected = ToggleButton(True, description='L1_1')
@interact(Selection=selected)
def modalpha(Selection):
    lH1.set_alpha(Selection)

selected2 = ToggleButton(True, description='L1_2')
@interact(Selection=selected2)
def modalpha2(Selection):
    lH2.set_alpha(Selection)

selected3 = ToggleButton(True, description='L1_3')
@interact(Selection=selected3)
def modalpha3(Selection):
    lH3.set_alpha(Selection)
    

In [ ]:
ToggleButton?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interact, FloatSlider, RadioButtons, ToggleButton

alpha_slider = ToggleButton(True)
# decorate the plot function with an environment from the UIs:
@interact(alpha=alpha_slider)
def plot(alpha):
    fig, ax = plt.subplots(figsize=(4, 3),
                       subplot_kw={'axisbg':'#EEEEEE',
                                   'axisbelow':True})
    ax.grid(color='w', linewidth=2, linestyle='solid')
    tme = a.data.currcsv.t
    mean = a.data.currcsv[["L1_1", "L1_2", "L1_3", "L1_4", "L1_5"]].mean(axis=1)
    lH = ax.plot(tme, mean, label="Mean", color=[0, 0, 0], lw=2, alpha=alpha)
    
    T1 = a.data.currcsv.L1_1
    lH = ax.plot(tme, T1, label="L1_1", color=[1, 0, 0], lw=1, alpha=alpha)
    

In [ ]:
from ipywidgets import interact, ToggleButton

interact(plot, color=ToggleButton(value = False))